<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Aguirre Orozco Miguel y Alvarado Garnica Luis

**Fecha:** 

**Expediente** : 
    
**Profesor:** Oscar David Jaramillo Zuluaga.

# Proyecto Módulo 2


# <span style="color:red"> MORTALIDAD POR INSUFICIENCIA CARDIACA

Las enfermedades cardiovasculares (ECV) son la principal causa de muerte a nivel mundial , cobrando aproximadamente 17,9 millones de vidas cada año , lo que representa el 31% de todas las muertes en todo el mundo .

La mayoría de las enfermedades cardiovasculares se pueden prevenir abordando los factores de riesgo conductuales como el consumo de tabaco, la dieta poco saludable y la obesidad, la inactividad física y el consumo nocivo de alcohol utilizando estrategias para toda la población.

Las personas con enfermedad cardiovascular o que tienen un alto riesgo cardiovascular (debido a la presencia de uno o más factores de riesgo como hipertensión, diabetes, hiperlipidemia o enfermedad ya establecida) necesitan una detección y manejo precoces en los que un modelo de aprendizaje automático puede ser de gran ayuda.

## OBJETIVOS

#### GENERAL

Predecir la muerte de una persona con insuficiencia cardiaca.

#### ESPECÍFICOS

* Encontrar la manera de complementar los nodos para realizar una predicción sobre muerte de cualquier persona. 
* Dar un apoyo extra a lo que los médicos puedan determinar con base a pruebas médicas.
* Detectar posibles muertes futuras y generar un cambio en la persona para que su salud pueda mejorar.
* Fungir como dato para la decisión del médico y paciente sobre operaciones, intervenciones o cualquier otro tratamiento que se deseé realizar.


## PROBLEMA

Se busca simular la muerte por insuficiencia cardiaca según los datos de distintos pacientes, tomando en cuenta sus pruebas médicas, historia clínica, hábitos y el flujo sanguíneo que presentan.

Todo esto se va a juntar en el periodo de observación que lleve el paciente, y según los datos se determina la probabilidad de muerte. 

<img src="imagen.png">
<img src="imagen2.png">

## NODOS Y VARIABLES

Pruebas médicas:
  * Importante conocer el estado actual del paciente y como está su sangre.
  
História clínica:
  * Una enfermadad como antecedente puede aumentar la probabilidad de muerte.
  
Hábitos:
  * El cuidado que le dan a su salud.
  
Flujo sanguíneo:
  * Cantidad de sangre que sale del corazón por cada contracción.
  
A todos estos nodos hay que agregarle el tiempo que se lleva en observación, es un determinante para aumentar o disminuir la probabilidad de muerte del paciente.

## HIPÓTESIS Y SUPUESTOS

Supuestos:
* Todos los datos son de pacientes con fallas cardiacas.
* Todas las muertes se dan por complicaciones con esa falla.
* Los resultados de los exámenes médicos son correctos.
* Cada una de las variables influye directamente en la muerte de una persona.

Hipótesis:
* Los hombres serán más propensos a morir debido a que su corazón es más grande en tamaño.
* Las personas con más edad morirán más fácilmente debido a un corazón más débil.
* Aquellos con un periodo de observación más prolongado tendrán oportunidad de mejorar su salud.

## BASE DE DATOS

La base de datos (autor: Davide Chicco) se obtiene por medio de la plataforma **Kaggle**, una comunidad en línea de científicos de datos y profesionales del aprendizaje automático que permite a los usuarios encontrar y publicar conjuntos de datos, explorar y crear modelos en un entorno de ciencia de datos basado en la web, trabajar con otros científicos de datos e ingenieros de aprendizaje automático y participar en concursos para resolver los desafíos de la ciencia de datos.

Enlace a base de datos: https://www.kaggle.com/andrewmvd/heart-failure-clinical-data